In [1]:
import cv2
import json
import numpy as np
import os
import os.path as osp
from time import time
from pathlib import Path
from tqdm import tqdm

from core import ObjectDetector, CornerPredictor

In [2]:
dt_vehicle = ObjectDetector('yolov4-default')
dt_plate   = ObjectDetector('yolov4-plate_type')
dt_number  = ObjectDetector('yolov4-plate_number')
dt_corner  = CornerPredictor('corner_prediction')

<yolov4-default> model loaded.
<yolov4-plate_type> model loaded.
<yolov4-plate_number> model loaded.
<corner_prediction> model loaded.


In [3]:
def start(img, pts):
    img_vehicle, img_plate, img_plate_warped = None, None, None
    plate, number = '', ''
    s_time = time()

    _ = dt_vehicle.loadImage(img)
    bbox1, _ = dt_vehicle.runDetection(mode='vehicle', multi_res=True)
    for box1 in bbox1:
        img_vehicle = dt_plate.loadImage(img, True, box1)
        bbox2, obj2 = dt_plate.runDetection(mode='plate')

        if len(bbox2) != 0:
            if (box1[0, 0]+bbox2[0, 0]) <= pts[0, 0] and (box1[0, 1]+bbox2[0, 1]) <= pts[0, 1] and\
            (box1[0, 0]+bbox2[1, 0]) >= pts[1, 0] and (box1[0, 1]+bbox2[1, 1]) >= pts[1, 1]:
                img_plate = dt_corner.loadImage(img_vehicle, True, bbox2)
                img_plate_warped, _ = dt_corner.runPrediction()

                _ = dt_number.loadImage(img_plate_warped)
                bbox3, obj3 = dt_number.runDetection(mode='number', multi_res=True)

                if len(bbox3) != 0:
                    plate = obj2
                    order = bbox3[:, 0, 0].argsort()
                    for o in order:
                        number += obj3[o]
                    break

    t_time = time() - s_time

    return t_time, plate, number, img_vehicle, img_plate, img_plate_warped

def error_log(p, label, predict, write=False, img1=None, img2=None, img3=None):
    log = {'path': p, 'label': label, 'predict': predict}
    
    if write:
        dir = 'error_log/system_error_ourDataset'
        file = osp.basename(p).split('-')[0]
        if not osp.exists(dir): os.mkdir(dir)
        cv2.imwrite(osp.join(dir, file+'.jpg'), cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        cv2.imwrite(osp.join(dir, file+'-plate.jpg'), cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        cv2.imwrite(osp.join(dir, file+'-plate_warped.jpg'), cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))

    return log

In [4]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\train_valid_data'
vehicle_dataset = osp.join(dataset_dir, 'vehicle_images')
plate_dataset = osp.join(dataset_dir, 'plate_images')

path_file = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\training_data\plate_number_detection\file_path.json'
with open(path_file, 'r') as f:
    test_path = json.load(f)['valid']

test_path = [osp.join(vehicle_dataset, osp.basename(p).split('-')[0]+'.jpg') for p in test_path]
print('total:', len(test_path))

total: 3496


In [5]:
n_test = len(test_path)
acc_list = [[n_test, 0], [n_test, 0], [n_test, 0]] # plate_ACC, number_ACC, p&n_ACC
total_time = 0
num_result = 0
sys_error = []

for i in tqdm(range(n_test), unit=' images', ncols=100):
    p = test_path[i]
    img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)

    plate_file = p.replace('jpg', 'json')
    with open(plate_file, 'r', encoding='utf-8') as f:
        data = json.load(f)['shapes'][0]
    plate = data['label']
    pts = np.array(data['points'])
    
    offset_x, offset_y = int(0.1*(pts[1, 0]-pts[0, 0])), int(0.1*(pts[1, 1]-pts[0, 1]))
    pts[0, 0], pts[0, 1] = pts[0, 0]+offset_x, pts[0, 1]+offset_y
    pts[1, 0], pts[1, 1] = pts[1, 0]-offset_x, pts[1, 1]-offset_y
    
    if i == 0:
        _, _, _, _, _, _ = start(img, pts)
    t_time, d_plate, d_number, img_vehicle, img_plate, img_plate_warped = start(img, pts)
    total_time += t_time

    if d_number != '':
        num_result += 1
        fn = osp.basename(p).split('.')[0]
        number_file = osp.join(plate_dataset, fn+'-plate.json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']] # plate numbers
        number = ''
        for n in nb:
            number += n

        if d_plate == plate:
            acc_list[0][1] += 1
        if d_number == number:
            acc_list[1][1] += 1
        else:
            sys_error.append(error_log(fn, [plate, number], [d_plate, d_number], True, img_vehicle, img_plate, img_plate_warped))
        if d_plate == plate and d_number == number:
            acc_list[2][1] += 1

print('Average time: {:.3f}s'.format(total_time/n_test))
print('ACC with no result:')
print('Plate: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0][0], acc_list[0][1], acc_list[0][1]/acc_list[0][0]*100))
print('Number: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[1][0], acc_list[1][1], acc_list[1][1]/acc_list[1][0]*100))
print('P and N: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[2][0], acc_list[2][1], acc_list[2][1]/acc_list[2][0]*100))
print('ACC without no result:')
print('Plate: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(num_result, acc_list[0][1], acc_list[0][1]/num_result*100))
print('Number: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(num_result, acc_list[1][1], acc_list[1][1]/num_result*100))
print('P and N: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(num_result, acc_list[2][1], acc_list[2][1]/num_result*100))

log_dict = {'sys_error': sys_error}
with open('error_log/system_error_ourDataset.json', 'w') as f:
    json.dump(log_dict, f, indent=2, ensure_ascii=False)

100%|██████████████████████████████████████████████████████| 3496/3496 [06:04<00:00,  9.60 images/s]

Average time: 0.095s
ACC with no result:
Plate: total=3496, correct=2242, accuracy=64.13%
Number: total=3496, correct=2220, accuracy=63.50%
P and N: total=3496, correct=2214, accuracy=63.33%
ACC without no result:
Plate: total=2249, correct=2242, accuracy=99.69%
Number: total=2249, correct=2220, accuracy=98.71%
P and N: total=2249, correct=2214, accuracy=98.44%
